<a href="https://colab.research.google.com/github/ChickenRob0t/W_Scrapping_Meteorologia/blob/main/Weather_link_SS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Instalar e importar todas las dependencias
!pip install selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime, timedelta

import pandas as pd
import time

In [ ]:
#Código para eliminar muchos problemas con Selenium (Esto sólo aplica en google colab)
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

#https://github.com/googlecolab/colabtools/issues/3347 Para recordar xd, sirve en mayo :p

In [ ]:
#Funciones que nos ayudan después en la ejecución del programa
def found_elements(x):
    return x.find_elements(By.TAG_NAME,'td')

def textit(x):
    return x.text

def To_celsius(x):
  if '°' in x:
    x=float(x.replace('°','').replace('F',''))
    return round((x-32)*5/9 , 1)
  else:
    return x

### Información inicial en variables
Aquí guardamos cosas como las páginas web con sus respectivas locaciones, la cuenta con la que entramos, etc.

In [ ]:
fecha_actual = datetime.now()                   #Problemas técnicos con la zona horaria para que me entregue el día de hoy siempre
desplazamiento_gdl = timedelta(hours=-6)

date_p= (fecha_actual + desplazamiento_gdl ).strftime('%Y-%m-%d')  #Esto entrega la fecha de hoy, no importa cuando lo corra.


cuenta='Robot_Chicken'   #Cuenta y contraseña de WeatherLink
pas='IAM123'


#Estaciones que nos interesan, si se necesita otra sólo usar la estructura que se tiene ('Nombre':[dirección url,'Nombre',Latitud,Longitud])
Estac={'Loma_bonita':['83359da8-44b9-4c74-afa6-a981ab96f837','Loma_bonita',20.639, -103.403],
'Nogalera': ['496b8d0c-90b5-405d-aee1-1c95195fb861','Nogalera',20.630, -103.342]}


#Columnas de la base de datos final
cols=["Name","Actual","Max_day","Hora_max","Min_day","Hora_min",'Estacion','Lat','Long']

In [ ]:
#Función que regresa las temperaturas y hora de una URL
def row(URL):
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("--no-sandbox")


    URL="https://www.weatherlink.com/bulletin/{}".format(URL)

    #Obtengo la información de la URL con las opciones predeterminadas
    driver = webdriver.Chrome(options=options)
    driver.get(URL)


    #En esta parte entro a la cuenta entregando usuario y contraseña
    cuent_in=driver.find_element(By.ID, 'username')
    pass_in=driver.find_element(By.ID, 'password')

    cuent_in.send_keys(cuenta)
    pass_in.send_keys(pas)

    boton=driver.find_element(By.ID,'submit')
    boton.click()

    #Entro en la parte de las tablas del programa y saco la información de ellas
    time.sleep(1)
    bot_2=driver.find_element(By.ID, 'summary-view')
    bot_2.click()

    tables=driver.find_elements(By.CLASS_NAME, 'wl-table')[1].find_element(By.TAG_NAME, 'tbody')
    filas=tables.find_elements(By.TAG_NAME, 'tr')
    col=filas[0].find_elements(By.TAG_NAME, 'td')


    #col=list(map(found_elements,filas))
    #xd=[]
    #for cols in col:                          Esto es por si se necesitara hacer de toda la tabla (la segunda)
       # xd.append(list(map(textit,cols)))       pero sólo nos interesa el valor de la temperatura
    #return xd

    return list(map(textit,col))

In [ ]:
DF_Temp=[]
Far=['Actual','Max_day','Min_day']

#Ciclo para cada estación que necesitemos (aprox 6s por estación )
for estacin in Estac:
    DF_Temp.append(row(Estac[estacin][0])+Estac[estacin][1:])

#La hacemos base de datos
DF_Temp=pd.DataFrame(DF_Temp,columns=cols)


In [ ]:
#Para cambiar las temperaturas de Farenheit a Celsius
for colum in Far:
  DF_Temp[colum]=DF_Temp[colum].apply(To_celsius)

DF_Temp

,Name,Actual,Max_day,Hora_max,Min_day,Hora_min,Estacion,Lat,Long
0,Temperature,27.2,34.4,4:45 PM,20.6,3:36 AM,Loma_bonita,20.639,-103.403
1,Temperature,30.0,31.7,12:35 PM,21.1,3:00 AM,Nogalera,20.630,-103.342


In [ ]:
DF_Temp.to_csv('{}_Weather_link.csv'.format(date_p))   #Guarda en formato csv